# Part 2, Topic 1: Introduction to Voltage Glitching (MAIN)

---
NOTE: This lab references some (commercial) training material on [ChipWhisperer.io](https://www.ChipWhisperer.io). You can freely execute and use the lab per the open-source license (including using it in your own courses if you distribute similarly), but you must maintain notice about this source location. Consider joining our training course to enjoy the full experience.

---

**SUMMARY:** *Similarly to clock glitching, inserting brief glitches into the power line of an embedded device can result in skipped instructions and corrupted results. Besides providing a more reliable glitch on some targets when compared to clock glitching, voltage glitching also has the advanatage that the Vcc pins on chips are always accessable. This won't be covered in this course, but it can also be used to glitch a device asynchronous to its clock.*

**LEARNING OUTCOMES:**

* Understanding voltage glitch settings
* Building a voltage glitch and crash map.
* Modifying glitch circuit to increase glitch success

## Voltage Glitch Hardware

The ChipWhisperer uses the same hardware block for both voltage and clock glitching, with the only difference being where the glitch output is routed to. Instead of routing to HS2, voltage glitching is performed by routing the glitch to either the `glitch_hp` transistor or the `glitch_lp` transistor. This can be done via the following API calls:

```python
scope.io.glitch_hp = True #enable HP glitch
scope.io.glitch_hp = False #disable LP glitch
scope.io.glitch_lp = True #enable LP glitch
scope.io.glitch_lp = False #disable LP glitch
```

While the hardware block are the same, you'll need to change how it's configued. You wouldn't want to try routing `"clock_xor"` to the glitch transistor and oscillate Vcc like the device's clock! Instead, the following two output settings are best suited to voltage glitching:

1. `"glitch_only"` - insert a glitch for a portion of a clock cycle based on `scope.glitch.width` and `scope.glitch.offset`
1. `"enable_only"` - insert a glitch for an entire clock cycle

Typically, the `"enable_only"` setting will be too powerful for most devices. One situation where it outshines `"glitch_only"` is in glitching asychronous to the target's clock. An example of this is glitching a target with an internal clock. In this case, the ChipWhisperer's clock can be boosted far above the target's to insert a precise glitch, with `repeat` functioning as `width` and `ext_offset` functioning as `offset`.

### Voltage Glitching vs. Clock Glitching

Voltage glitching has some obvious benefits over clock glitching, such as working for a wider varitey of targets, but its downsides are less obvious. One of the biggest is how much it depends on the actual glitch circuit itself. With clock glitching, it's relatively easy to insert a glitch - there's nothing external trying to keep the clock at a certain voltage level. This is very different for a target's power pins. When we try to drop the power pin to ground, there's a lot of stuff fighting us to keep the power pin at the correct voltage, such as decoupling capacitors, bulk supply capacitors, and the voltage regulator supplying the voltage. This means when we make small changes to the glitch circuit, the glitch settings and even our ability to insert a glitch at all completely change! Consider glitching a target on the CW308 UFO board. If you switch your coaxial cable length from 20cm to 40 cm, you'll need to find entirely new glitch settings to repeat the attack (if it's still even possible). This is quite easy to see on an oscilloscope or  using the ChipWhisperer's ADC: longer cables and lower valued shunt resistors will make the glitch less sharp and increase ringing.

While your first thought might be to go for as sharp a glitch as possible, this often won't result in a high glitch success rate. If you're unable to find any working glitches with your current setup, it might be worth changing you hardware setup a bit. For example, on the ChipWhisperer Lite 1 part, you can desolder SJ5 and solder header pins to JP6. Even just connecting these pins with a jumper will have different glitch behaviour than with a soldered SJ5.

You can refer to the training slides for more information about finding good glitch settings, as well as more on the theory side of voltage glitching.

### The Lab

To introduce you to volatge glitching and find some settings, we're going to walk back through the clock glitching loop lab. You may want to capture some power traces while you're first experimenting with glitches to see what effects different glitch widths have on the power trace. Another thing to keep in mind is that targets often won't tolerate the Vcc pin dropping for an extended period of time without crashing - once you see the target start to crash, you won't see much else with larger widths.

One thing you might have to change is the glitch repeat value. Depending on how wide your glitch is, the voltage at the power pin may not recover by the time the next glitch is inserted. This can have to effect of increasing subsequent glitches' strength, which may or may not be desirable. Since glitches inserted with repeat > 1 have different strength, it's a good idea to scan through ext_offset as well.

###  Higher Frequency Glitching

The XMEGA target, and to a lesser extent the STM32F3, is very difficult to glitch with the default ChipWhisperer settings. Try bumping the clock frequency to 24MHz for the STM32 or 32MHz for the XMEGA and use a repeat 5-10 with both the high power and low power glitches active. You'll need to adjust the baud rate by the same proportion as the clock. This will increase the glitch precision, as the glitch width/offset step is based on the target clock, and may decrease the stability of the target.

Another setup that seems to work with the XMEGA is SJ5 unsoldered, JP6 jumpered, high+low power glitch, 32MHz, and repeat=5.

The ChipWhisperer Husky, with its PLL based glitching system, can typically glitch targets without needing to change the target's frequency.

### Disabling Logging

When glitching (or just running normally in earlier labs), you may have seen various warnings from loggers ChipWhisperer uses. This often has useful information, especially if things don't work right, but for voltage 
glitching especially, it mostly clutters up any print output you have. As such, we'll disable logging for the voltage glitching labs:

```python
cw.set_all_log_levels(cw.logging.CRITICAL)
```

You can reenable logging via

```python
cw.set_all_log_levels(cw.logging.WARNING)
```

In [1]:
SCOPETYPE = 'OPENADC'
PLATFORM = 'CWHUSKY'
allowable_exceptions = None

#!/usr/bin/env python
# coding: utf-8

# In[ ]:


import chipwhisperer as cw

try:
    if not scope.connectStatus:
        scope.con()
except NameError:
    scope = cw.scope()

try:
    if SS_VER == "SS_VER_2_1":
        target_type = cw.targets.SimpleSerial2
    elif SS_VER == "SS_VER_2_0":
        raise OSError("SS_VER_2_0 is deprecated. Use SS_VER_2_1")
    else:
        target_type = cw.targets.SimpleSerial
except:
    SS_VER="SS_VER_1_1"
    target_type = cw.targets.SimpleSerial

try:
    target = cw.target(scope, target_type)
except:
    print("INFO: Caught exception on reconnecting to target - attempting to reconnect to scope first.")
    print("INFO: This is a work-around when USB has died without Python knowing. Ignore errors above this line.")
    scope = cw.scope()
    target = cw.target(scope, target_type)


print("INFO: Found ChipWhisperer😍")


prog = cw.programmers.STM32FProgrammer


import time
time.sleep(0.05)
scope.default_setup()





INFO: Found ChipWhisperer😍
scope.gain.gain                          changed from 0                         to 22                       
scope.gain.db                            changed from 15.0                      to 25.091743119266056       
scope.adc.samples                        changed from 131124                    to 5000                     
scope.clock.clkgen_freq                  changed from 0                         to 7363636.363636363        
scope.clock.adc_freq                     changed from 0                         to 29454545.454545453       
scope.clock.adc_rate                     changed from 0.0                       to 29454545.454545453       
scope.io.tio1                            changed from serial_tx                 to serial_rx                
scope.io.tio2                            changed from serial_rx                 to serial_tx                
scope.io.hs2                             changed from None                      to clkgen            

In [10]:


target.baud = 115200
##set trigger out on mcx connector

#scope.io.glitch_trig_mcx = "glitch"
scope.io.glitch_trig_mcx = "trigger"


    
scope.clock.clkgen_src = 'system'
scope.clock.clkgen_freq = 200e6
scope.clock.adc_mul = 1

scope.adc.basic_mode = "rising_edge"

scope.trigger.triggers = "tio4" # Trigger on rising edge of IO4

scope.glitch.enabled = True
scope.glitch.clk_src = "pll"
scope.clock.pll.update_fpga_vco(600e6)
scope.glitch.output = 'enable_only' #glitch only doesnt seem to work
scope.glitch.trigger_src = "ext_single" #fire everytime trigger is tripped
#cannot use ext continuous if relying on target signals cause 

scope.glitch.offset = 1000
scope.glitch.width = 1000
scope.glitch.ext_offset = 25
scope.glitch.repeat = 500#40 when hot, 41 when cold



scope.io.glitch_lp = False
scope.io.glitch_hp = True
#for async glitching, crank the clock speed all the way up to 200mhz, can try overclocking also too
#for some reason the glitch width changes depending on whether the LP mos is active or not
#HP only: 500 clock cycles (repeat) ->3.440uS
#LP only: 500 clock cycles (repeat) ->3.940uS

#BOTH: 500 clock cycles (repeat) ->3.940uS
#100mhz HP only->5.080uS
#100mhz HP+LP -> 5.520uS
#100mhz hp+lp 100 cycles -> 1.5uS
#100mhz hp only 100 cycles -> 1.000uS

#choose HP mos only for best timing accuracy (scope.glitch.repeat/scope.clock.clkgen_freq)
#offset 0: 128nS latency
#offset 50: 624ns latency
#offset from trigger = (scope.glitch.ext_offset/scope.clock.clkgen_freq)
scope.adc.lo_gain_errors_disabled = True
scope.adc.clip_errors_disabled = True

scope.arm()             # Arm the glitch + ADC modules
#print(scope.glitch)
#print(scope.adc)
#print(scope.clock)

# Set up UART
target.baud = 115200

def reboot_flush():
    scope.glitch.repeat = 500
    scope.glitch.trigger_src = "manual"
    scope.arm()
    #print(scope.errors)
    scope.errors.clear()
    scope.glitch.trigger_src = "ext_single" #fire everytime trigger is tripped
    target.flush()
    time.sleep(0.1)



In [ ]:
#use for manual sweeps
import ipywidgets as widgets
from IPython.display import display
import threading
import time

# ------------------------------
# UI widgets
# ------------------------------
repeat_slider = widgets.IntSlider(description='Repeat', min=5, max=500, value=20)
offset_slider = widgets.IntSlider(description='Ext Offset', min=0, max=500, value=25)
attack_button = widgets.ToggleButton(description="Start Attack", value=False)

# Display UI
display(repeat_slider, offset_slider, attack_button)

# ------------------------------
# Slider event -> update glitch settings
# ------------------------------
def update_glitch_settings(change=None):
    scope.glitch.repeat = repeat_slider.value
    scope.glitch.ext_offset = offset_slider.value

# Connect slider events
repeat_slider.observe(update_glitch_settings, names='value')
offset_slider.observe(update_glitch_settings, names='value')

# Manually apply initial settings
update_glitch_settings()

# ------------------------------
# Button event -> enable/disable glitch
# ------------------------------
def on_attack_toggle(change):
    if change["new"]:
        print("⚡ Glitch attack ENABLED")
        update_glitch_settings()
        scope.glitch.enabled = True
    else:
        print("🛑 Glitch attack DISABLED")
        scope.glitch.enabled = False

attack_button.observe(on_attack_toggle, names="value")

# ------------------------------
# UART logger thread
# ------------------------------
def uart_logger():
    while True:
        try:
            while target.in_waiting():
                data = target.read()
                print(data, end='')  # no decoding, already string
        except Exception as e:
            print("UART error:", e)
        time.sleep(0.1)

# Set up UART
target.baud = 115200

"""
# Start logger thread once
if 'uart_thread' not in globals():
    uart_thread = threading.Thread(target=uart_logger, daemon=True)
    uart_thread.start()
   """ 



In [22]:
#send g, wait for trigger, glitch, wait for result, check if result

#gc.set_step("ext_offset", 5)
#gc.set_range("width", 2400, 2700)
#gc.set_range("offset", 2400, 2700)

gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["repeat", "ext_offset", "tries"])
gc.display_stats()


num_tries = 10 # increase to get better glitch stats
gc.set_global_step([1]) # reduce to fine tune glitching
    

gc.set_range("ext_offset", 100, 8191)
gc.set_range("repeat", 91, 91)
gc.set_range("tries", 1, num_tries)
gc.set_step("repeat", 1)
gc.set_step("ext_offset", 1)
gc.set_step("tries", 1)
scope.glitch.enabled = True

for glitch_setting in gc.glitch_values():
    if scope.adc.state:
        #Device is slow to boot?
        reboot_flush()
        time.sleep(0.1)
        
   
    scope.glitch.repeat = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[1]
    scope.arm()
    target.write("g")
    val = target.read(15, 10).strip()#For loop check
    #print(val)
    normal = (val == "r2500")
    glitch = val.startswith("r") and val[1:].isdigit() and int(val[1:]) != 2500


    
    if glitch is True:
        gc.add("success")
        print((scope.glitch.repeat, scope.glitch.ext_offset, val))
    elif normal is True:
        gc.add("normal")
    else:
        gc.add("reset")
        reboot_flush()

print("Done glitching")
#enable logging
cw.set_all_log_levels(cw.logging.WARNING)

IntText(value=0, description='success count:', disabled=True)

IntText(value=0, description='reset count:', disabled=True)

IntText(value=0, description='normal count:', disabled=True)

FloatSlider(value=0.0, continuous_update=False, description='repeat setting:', disabled=True, max=10.0, readou…

FloatSlider(value=0.0, continuous_update=False, description='ext_offset setting:', disabled=True, max=10.0, re…

...

(91, 5218, 'r2497')
(91, 5218, 'r2497')
(91, 5218, 'r2497')
(91, 5218, 'r2497')
(91, 5218, 'r2497')
(91, 5219, 'r2497')
(91, 5219, 'r2497')
(91, 5219, 'r2497')
(91, 5219, 'r2497')
(91, 5219, 'r2450')
(91, 5219, 'r2497')
(91, 5219, 'r2497')
(91, 5220, 'r2497')
(91, 5220, 'r2497')
(91, 5220, 'r2497')
(91, 5220, 'r2497')
(91, 5220, 'r2497')
(91, 5220, 'r2514')
(91, 5221, 'r958590650')
(91, 5229, 'r4294967295')
(91, 5253, 'r2450')
(91, 5254, 'r2450')
(91, 5254, 'r2450')
(91, 5254, 'r2450')
(91, 5254, 'r2450')
(91, 5254, 'r2450')
(91, 5254, 'r2450')
(91, 5255, 'r2450')
(91, 5255, 'r2450')
(91, 5255, 'r2450')
(91, 5270, 'r2498')
(91, 5270, 'r2498')
(91, 5271, 'r2498')
(91, 5271, 'r2498')
(91, 5271, 'r2498')
(91, 5271, 'r2517')
(91, 5271, 'r2498')
(91, 5271, 'r2498')
(91, 5272, 'r2515')
(91, 5272, 'r2498')
(91, 5272, 'r2498')
(91, 5272, 'r2498')
(91, 5272, 'r2498')
(91, 5272, 'r2498')
(91, 5272, 'r2498')
(91, 5273, 'r2498')
(91, 5273, 'r2517')
(91, 5273, 'r2498')
(91, 5273, 'r2498')
(91, 5273

(91, 5931, 'r2459')
(91, 5932, 'r2459')
(91, 5940, 'r4294967295')
(91, 5940, 'r4294967295')
(91, 5960, 'r2400')
(91, 5965, 'r2400')
(91, 5965, 'r2400')
(91, 5965, 'r2400')
(91, 5980, 'r2456')
(91, 5981, 'r2460')
(91, 5982, 'r2477')
(91, 5982, 'r2461')
(91, 5982, 'r2460')
(91, 5982, 'r2460')
(91, 5982, 'r2460')
(91, 5983, 'r2460')
(91, 5983, 'r2460')
(91, 5983, 'r2460')
(91, 5983, 'r2460')
(91, 5983, 'r2460')
(91, 5983, 'r2460')
(91, 5983, 'r2460')
(91, 5983, 'r2460')
(91, 5984, 'r2477')
(91, 5984, 'r2460')
(91, 5984, 'r2460')
(91, 5984, 'r2460')
(91, 5984, 'r2460')
(91, 5984, 'r2460')
(91, 5984, 'r2460')
(91, 5985, 'r2460')
(91, 5985, 'r2460')
(91, 5994, 'r4294967295')
(91, 6017, 'r2400')
(91, 6017, 'r2400')
(91, 6018, 'r2400')
(91, 6018, 'r2400')
(91, 6018, 'r2400')
(91, 6018, 'r2400')
(91, 6019, 'r2400')
(91, 6019, 'r2400')
(91, 6019, 'r2400')
(91, 6034, 'r2461')
(91, 6034, 'r2478')
(91, 6035, 'r2478')
(91, 6035, 'r2478')
(91, 6035, 'r2478')
(91, 6035, 'r2478')
(91, 6036, 'r2478')
(9

(91, 6628, 'r4294967295')
(91, 6629, 'r4294967295')
(91, 6629, 'r4294967295')
(91, 6652, 'r2400')
(91, 6653, 'r2400')
(91, 6654, 'r2400')
(91, 6663, 'r2469')
(91, 6669, 'r2469')
(91, 6670, 'r2473')
(91, 6670, 'r2473')
(91, 6670, 'r2473')
(91, 6670, 'r2473')
(91, 6670, 'r2473')
(91, 6671, 'r2473')
(91, 6671, 'r2473')
(91, 6671, 'r2492')
(91, 6671, 'r2473')
(91, 6671, 'r2473')
(91, 6671, 'r2473')
(91, 6672, 'r2473')
(91, 6672, 'r2473')
(91, 6672, 'r2473')
(91, 6672, 'r2473')
(91, 6672, 'r2473')
(91, 6672, 'r2490')
(91, 6672, 'r2490')
(91, 6672, 'r2490')
(91, 6672, 'r2473')
(91, 6673, 'r2473')
(91, 6673, 'r2473')
(91, 6673, 'r2492')
(91, 6673, 'r2473')
(91, 6673, 'r2473')
(91, 6673, 'r2473')
(91, 6673, 'r2473')
(91, 6674, 'r2473')
(91, 6674, 'r2473')
(91, 6674, 'r2473')
(91, 6681, 'r4294967295')
(91, 6705, 'r2400')
(91, 6705, 'r2400')
(91, 6706, 'r2400')
(91, 6706, 'r2400')
(91, 6707, 'r2400')
(91, 6707, 'r2400')
(91, 6707, 'r2400')
(91, 6707, 'r2400')
(91, 6707, 'r2400')
(91, 6708, 'r240

(91, 7361, 'r2486')
(91, 7361, 'r2486')
(91, 7361, 'r690153146')
(91, 7362, 'r2486')
(91, 7362, 'r2486')
(91, 7362, 'r2486')
(91, 7372, 'r4294967295')
(91, 7394, 'r2400')
(91, 7394, 'r2400')
(91, 7395, 'r2400')
(91, 7396, 'r2400')
(91, 7396, 'r2400')
(91, 7396, 'r2400')
(91, 7412, 'r2487')
(91, 7412, 'r2487')
(91, 7412, 'r2504')
(91, 7413, 'r2506')
(91, 7413, 'r2487')
(91, 7413, 'r2487')
(91, 7413, 'r2504')
(91, 7413, 'r2487')
(91, 7413, 'r2487')
(91, 7413, 'r2504')
(91, 7414, 'r2487')
(91, 7414, 'r2487')
(91, 7414, 'r2487')
(91, 7414, 'r2487')
(91, 7414, 'r2487')
(91, 7414, 'r2487')
(91, 7415, 'r958457530')
(91, 7415, 'r690153147')
(91, 7415, 'r2487')
(91, 7415, 'r2487')
(91, 7415, 'r2487')
(91, 7423, 'r4294930175')
(91, 7447, 'r2400')
(91, 7448, 'r2400')
(91, 7448, 'r2400')
(91, 7450, 'r2400')
(91, 7456, 'r2484')
(91, 7465, 'r2488')
(91, 7466, 'r2488')
(91, 7466, 'r2488')
(91, 7466, 'r2488')
(91, 7467, 'r2488')
(91, 7467, 'r2488')
(91, 7467, 'r2488')
(91, 7467, 'r2488')
(91, 7467, 'r

(91, 8102, 'r2517')
(91, 8102, 'r2517')
(91, 8102, 'r2519')
(91, 8103, 'r2517')
(91, 8103, 'r2519')
(91, 8104, 'r690022075')
(91, 8109, 'r2501')
(91, 8110, 'r2501')
(91, 8110, 'r2501')
(91, 8110, 'r2501')
(91, 8111, 'r2501')
(91, 8111, 'r2501')
(91, 8111, 'r2501')
(91, 8111, 'r2501')
(91, 8112, 'r2501')
(91, 8112, 'r2501')
(91, 8112, 'r2501')
(91, 8112, 'r2501')
(91, 8112, 'r2501')
(91, 8113, 'r2501')
(91, 8125, 'r0150')
(91, 8126, 'r0150')
(91, 8126, 'r0150')
(91, 8127, 'r0150')
(91, 8127, 'r0150')
(91, 8127, 'r0150')
(91, 8127, 'r0150')
(91, 8128, 'r0150')
(91, 8128, 'r0150')
(91, 8128, 'r0150')
(91, 8128, 'r0150')
(91, 8128, 'r0150')
(91, 8128, 'r0150')
(91, 8136, 'r2550')
(91, 8136, 'r2550')
(91, 8136, 'r2550')
(91, 8137, 'r2550')
(91, 8137, 'r2550')
(91, 8138, 'r2550')
(91, 8138, 'r2550')
(91, 8138, 'r2550')
(91, 8146, 'r2497')
(91, 8148, 'r2497')
(91, 8174, 'r2450')
(91, 8174, 'r2450')
(91, 8175, 'r2450')
(91, 8175, 'r2450')
(91, 8175, 'r2450')
(91, 8176, 'r2450')
(91, 8177, 'r24

In [ ]:
#send g, wait for trigger, glitch, wait for result, check if result

gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["repeat", "ext_offset", "tries"])
gc.display_stats()


num_tries = 5 # increase to get better glitch stats
gc.set_range("tries", 1, 1)
#gc.set_range("width", 2400, 2700)
#gc.set_range("offset", 2400, 2700)
gc.set_global_step([20]) # reduce to fine tune glitching
    
#gc.set_step("ext_offset", 5)
gc.set_range("ext_offset", 0, 8191)
gc.set_range("repeat", 5, 100)


for glitch_setting in gc.glitch_values():
    scope.glitch.repeat = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[1]
    time.sleep(0.01)
    

In [ ]:
scope.glitch.repeat = 500
scope.glitch.trigger_src = "manual"
scope.arm()
#print(scope.errors)
scope.errors.clear()
scope.glitch.trigger_src = "ext_continuous" #fire everytime trigger is tripped

In [17]:
import time

try:
    while True:
        target.write("g")
        time.sleep(0.01)
except KeyboardInterrupt:
    print("Loop stopped.")


Loop stopped.


In [ ]:

##set trigger out on mcx connector

#scope.io.glitch_trig_mcx = "glitch"
scope.io.glitch_trig_mcx = "trigger"


    
scope.clock.clkgen_src = 'system'
scope.clock.clkgen_freq = 10e6
scope.clock.adc_mul = 1

scope.adc.basic_mode = "rising_edge"

scope.trigger.triggers = "tio4" # Trigger on rising edge of IO4

scope.glitch.enabled = True
scope.glitch.clk_src = "pll"
scope.io.glitch_hp = True
scope.io.glitch_lp = False
    


In [ ]:
reboot_flush()

scope.glitch.ext_offset = 25
scope.glitch.repeat = 500

scope.arm()
target.write("g")
scope.capture()
val = target.read(5, 10)#For loop check
normal = (val == "r2500")

glitch = val.startswith("r") and val[1:].isdigit() and int(val[1:]) != 2500

print("Normal:", normal)
print("Glitch:", glitch)
print("Value:", val)

In [ ]:
gc = cw.GlitchController(groups=["success", "reset", "normal"], parameters=["width", "offset", "ext_offset", "tries"])
gc.display_stats()

In [ ]:
gc.glitch_plot(plotdots={"success":"+g", "reset":"xr", "normal":None}, bufferlen=int(10E6))

Next, we'll setup the glitch. The main differences here compared to clock glitching is the use of `scope.io.glitch_lp/hp`, which activates the transistors used for voltage, and `scope.glitch.output = 'glitch_only'`, which sends the glitch output to the transistors instead of xoring it with the clock like with clock glitching.

In [ ]:
if scope._is_husky:
    scope.glitch.enabled = True
    scope.glitch.clk_src = "pll"
    scope.io.glitch_hp = True
    scope.io.glitch_lp = False
else:
    scope.glitch.clk_src = "clkgen" # set glitch input clock
scope.glitch.output = "glitch_only" # glitch_out = clk ^ glitch
scope.glitch.trigger_src = "ext_single" # glitch only after scope.arm() called
if PLATFORM == "CWLITEXMEGA":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CWLITEARM":
    scope.io.glitch_lp = True
    scope.io.glitch_hp = True
elif PLATFORM == "CW308_STM32F3":
    scope.io.glitch_hp = True
    scope.io.glitch_lp = True

Some tips for finding good glitches:

1. There's a lot of stuff fighting our glitch this time - unlike the clock line, the Vcc rail isn't supposed to oscillate! As such shorter glitches will have no effect. Often, good widths will be just below when the target starts consistantly crashing, so a good strategy is to find the minimum width where crashes always happen, then backing the width off a bit.
1. The repeat parameter behaves very differently than with voltage glitching - at the boosted clock rate, the Vcc often won't recover before the next glitch. Try different repeat values as well.
1. We've built in a success/reset measurement into the glitch loop. Once you've found some glitch spots, this will help you evaluate which ones are best for your target.

It can take a very long time to do go through the full search space, so you may want to stop after you get a certain number of succeses. By default here, it will be 1, but you may want to change it to 10, 20, or go even higher.

In [ ]:
MAX_SUCCESSES = 100000

num_tries = 5 # increase to get better glitch stats
gc.set_range("tries", 1, num_tries)
if scope._is_husky:
    gc.set_range("width", 2400, 2700)
    gc.set_range("offset", 2400, 2700)
    gc.set_global_step([10]) # reduce to fine tune glitching
    
    gc.set_step("ext_offset", 1)
    gc.set_range("ext_offset", 10, 50)
    scope.glitch.repeat = 1
    scope.adc.lo_gain_errors_disabled = True
    scope.adc.clip_errors_disabled = True
elif PLATFORM=="CWLITEXMEGA":
    gc.set_range("width", 43.5, 47.8)
    gc.set_range("offset", -48, -10)
    gc.set_range("ext_offset", 7, 10)
    gc.set_global_step(0.4)
    gc.set_step("ext_offset", 1)
    scope.glitch.repeat = 11
elif PLATFORM == "CWLITEARM":
    #should also work for the bootloader memory dump
    gc.set_range("width", 34, 36)
    gc.set_range("offset", -40, 10)
    gc.set_range("ext_offset", 4, 30)
    gc.set_global_step(0.4)
    gc.set_step("ext_offset", 1)
    scope.glitch.repeat = 7
elif PLATFORM == "CW308_STM32F3":
    #these specific settings seem to work well for some reason
    #also works for the bootloader memory dump
    gc.set_range("ext_offset", 4, 30)
    gc.set_range("width", 47.6, 49.6)
    gc.set_range("offset", -19, -21.5)
    gc.set_global_step(0.4)
    gc.set_step("ext_offset", 1)
    scope.glitch.repeat = 5

gc.set_step("tries", 1)

In [ ]:
import struct

#disable logging
cw.set_all_log_levels(cw.logging.CRITICAL)

scope.adc.timeout = 0.5

reboot_flush()
loff = scope.glitch.offset
lwid = scope.glitch.width
total_successes = 0
successes = 0
resets = 0
for glitch_setting in gc.glitch_values():
    scope.glitch.offset = glitch_setting[1]
    scope.glitch.width = glitch_setting[0]
    scope.glitch.ext_offset = glitch_setting[2]
    #print(scope.glitch.ext_offset)
    if glitch_setting[3] == 1:
        total_successes += successes
        if (successes > 0):
            print("successes = {}, resets = {}, offset = {}, width = {}, ext_offset = {}".format(successes, resets, scope.glitch.offset, scope.glitch.width, scope.glitch.ext_offset))
            total_successes += successes
        successes = 0
        resets = 0
        #if total_successes > MAX_SUCCESSES:
        #    break
    target.flush() #Removes all data from the serial buffer.
    if scope.adc.state:#The current state of the trigger input. can detect crash here (fast) before timing out (slow)
        #print("Trigger still high!")
        gc.add("reset")

        #Device is slow to boot?
        reboot_flush()
        resets += 1

    scope.arm()

    #Do glitch loop
    target.write("g")

    ret = scope.capture()

    scope.io.vglitch_reset()
    if ret:
        #print('Timeout - no trigger')
        gc.add("reset")
        resets += 1

        #Device is slow to boot?
        reboot_flush()

    else:
        val = target.simpleserial_read_witherrors('r', 4, glitch_timeout=10, timeout=50)#For loop check
        if val['valid'] is False:
            gc.add("reset")
            reboot_flush()
            resets += 1
            #print(val)
        else:
            gcnt = struct.unpack("<I", val['payload'])[0]

            if gcnt != 2500: #for loop check
                gc.add("success")
                #print((scope.glitch.width, scope.glitch.offset, scope.glitch.ext_offset))
                successes += 1
            else:
                gc.add("normal")

print("Done glitching")
#enable logging
cw.set_all_log_levels(cw.logging.WARNING)

With that done, let's print our results. You'll likely want to ignore the "tries" and "ext_offset" parameters:

In [ ]:
results = gc.calc(ignore_params=["tries", "ext_offset"], sort="success_rate")
results

In [ ]:
gc.plot_2d(alpha=False)

In [ ]:
scope.dis()
target.dis()

In [ ]:
assert total_successes >= 1

In [9]:
target.write("g")
